> A estratégia consiste em selecionar os business que aparecem em eval_set, selecionar as 5 melhores reviews dele e as 5 piores reviews (com menor nota), gerar embeddings dessas reviews e calcular a média.

> Depois, fazemos o mesmo para os users que aparecem em eval_set: pegamos as 5 melhores, 5 piores, tira a média dos embeddings e comparamos com as médias dos business.

> A lógica é: se os embeddings de melhor avaliação for maior, recomenda para a pessoa, se o de pior avaliação for maior, não recomenda.

> Para ordenar os business, verificamos qual é maior:
- se for o embedding de melhor avaliação, ordenamos de forma que quanto maior o valor dele, mais à frente fica.
- se for o embedding de pior avaliação, ordenamos de forma que quanto maior o valor dele, mais atrás ele fica.

In [2]:
import pandas as pd

# Carregando os dados

In [3]:
# carregando os dados do eval_set
eval_set = pd.read_csv('../data/evaluation/eval_users.csv')

In [4]:
# carregando o dataset de reviews
reviews = pd.read_parquet('../data/DatasetsLimpos/yelp_academic_dataset_review.parquet')

# Filtrando

### Filtrando apenas os business que aparecem no eval_set

In [5]:
## carregando eval_df
# verificando os business que aparecem em eval
business_ = eval_set['gt_reclist'].apply(lambda x: x.replace("[", "").replace("]", "").split(',')).explode().unique()

In [6]:
business_ = pd.Series(business_).apply(lambda x: x.replace("'", "").replace(" ", "")).unique()

In [7]:
# filtrando apenas os business que aparecem em eval
reviews = reviews[reviews['business_id'].isin(business_)]

In [8]:
reviews.shape

(1935414, 7)

In [9]:
# selecionando apenas as colunas que serão utilizadas
reviews = reviews[['business_id', 'user_id','text', 'stars']]

In [14]:
# agrupando os business
reviews = reviews.groupby('business_id').agg({'text': lambda x: [i for i in x], 'stars': lambda x: [i for i in x]})

In [26]:
import numpy as np

In [30]:
# criando uma função que recebe uma lista e retorna os índices dos 5 maiores e 5 menores valores (ou 2)
def get_best_worst_indices(lista):
    # Converte a lista para um array NumPy
    arr = np.array(lista)

    if arr.shape[0] < 10:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:2]
        indices_n_menores = np.argsort(arr)[:2]
    else:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:5]
        indices_n_menores = np.argsort(arr)[:5]
    return indices_n_maiores.tolist(), indices_n_menores.tolist()

In [ ]:
# aplicando na coluna stars
reviews['stars'] = reviews.stars.apply(get_best_worst_indices)

In [ ]:
# filtrando a coluna texto para pegar apenas as reviews que estão nos indices de stars
reviews['text'] = reviews.apply(lambda x: [x['text'][i] for i in x['stars'][0] + x['stars'][1]], axis=1)

### Gerando os embeddings dos business

In [41]:
from BertEmbedding import get_bert_embedding

In [ ]:
# aplicando a funcao no text
reviews['embs'] = reviews['text'].apply(lambda x: get_bert_embedding(x)[0])

In [47]:
# separando os embeddings em colunas: melhor avaliados [:5] e piores avaliados [5:]
reviews['embs_best'] = reviews['embs'].apply(lambda x: x[:5] if x.shape[0] == 10 else x[:2])
reviews['embs_worst'] = reviews['embs'].apply(lambda x: x[5:] if x.shape[0] == 10 else x[2:])

In [67]:
reviews.head()

,text,stars,embs,embs_best,embs_worst
business_id,,,,,
--onnLZrsCazmcy2P_7fcw,[Another place where I'm a regular which means...,"([0, 2], [1, 4])","[[1.0513206, -0.8850983, 2.8266966, -1.4233696...","[[1.0513206, -0.8850983, 2.8266966, -1.4233696...","[[0.7261548, -0.94964, 2.6089482, -1.1552327, ..."
--x_BmZbxzK_nx_GHBaRVw,"[This is a fun, fun tour, and Johnathan is a g...","([22, 15, 18, 13, 12], [2, 0, 14, 11, 3])","[[0.6877591, -0.8208134, 2.5717452, -0.9616006...","[[0.6877591, -0.8208134, 2.5717452, -0.9616006...","[[0.40403867, -1.040651, 2.6346776, -1.0031796..."
-1B9pP_CrRBJYPICE5WbRA,[I ordered the Singapore St Noodle with medium...,"([425, 362, 716, 717, 357], [583, 582, 633, 73...","[[1.395613, -0.88007987, 3.027491, -1.2700746,...","[[1.395613, -0.88007987, 3.027491, -1.2700746,...","[[0.16083443, -0.68887424, 2.4779906, -0.98172..."
-1MhPXk1FglglUAmuPLIGg,[Great food and drink. The staff are always fr...,"([0, 34, 38, 44, 47], [99, 92, 37, 3, 73])","[[1.2828859, -0.8165944, 3.0964985, -1.3718348...","[[1.2828859, -0.8165944, 3.0964985, -1.3718348...","[[0.26243037, -0.6838061, 2.2511733, -1.038180..."
-1b2kNOowsPrPpBOK4lNkQ,[Love this place! We went because it was walki...,"([0, 19, 18, 17, 16], [10, 5, 3, 9, 1])","[[0.8386996, -1.0572733, 2.4753923, -1.0825523...","[[0.8386996, -1.0572733, 2.4753923, -1.0825523...","[[0.6118287, -0.7351066, 2.3257384, -0.7784133..."


In [78]:
# gerando media dos embeddings
reviews['embs_best'] = reviews.embs_best.apply(lambda x: np.mean(x, axis=0))
reviews['embs_worst'] = reviews.embs_worst.apply(lambda x: np.mean(x, axis=0))

In [79]:
reviews[['embs_best', 'embs_worst']].to_parquet('../data/embeddingsBusiness.parquet')

### Filtrando os users que aparecem em eval_set

In [82]:
users = eval_set['user_id'].unique()

In [83]:
# carregando o dataset de reviews
reviews = pd.read_parquet('../data/DatasetsLimpos/yelp_academic_dataset_review.parquet')

In [88]:
reviews = reviews[reviews['user_id'].isin(users)]

In [90]:
## Todo_ o resto é igual...

# selecionando apenas as colunas que serão utilizadas
reviews = reviews[['user_id', 'text', 'stars']]

# agrupando os business
reviews = reviews.groupby('user_id').agg({'text': lambda x: [i for i in x], 'stars': lambda x: [i for i in x]})

# criando uma função que recebe uma lista e retorna os índices dos 5 maiores e 5 menores valores (ou 2)
def get_best_worst_indices(lista):
    # Converte a lista para um array NumPy
    arr = np.array(lista)

    if arr.shape[0] < 10:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:2]
        indices_n_menores = np.argsort(arr)[:2]
    else:
        # Obtém os índices dos N maiores valores usando argsort
        indices_n_maiores = np.argsort(-arr)[:5]
        indices_n_menores = np.argsort(arr)[:5]
    return indices_n_maiores.tolist(), indices_n_menores.tolist()


# aplicando na coluna stars
reviews['stars'] = reviews.stars.apply(get_best_worst_indices)
# filtrando a coluna texto para pegar apenas as reviews que estão nos indices de stars
reviews['text'] = reviews.apply(lambda x: [x['text'][i] for i in x['stars'][0] + x['stars'][1]], axis=1)

### Gerando os embeddings dos user_id

In [ ]:
# aplicando a funcao no text
reviews['embs'] = reviews['text'].apply(lambda x: get_bert_embedding(x)[0])

In [92]:
# separando os embeddings em colunas: melhor avaliados [:5] e piores avaliados [5:]
reviews['embs_best'] = reviews['embs'].apply(lambda x: x[:5] if x.shape[0] == 10 else x[:2])
reviews['embs_worst'] = reviews['embs'].apply(lambda x: x[5:] if x.shape[0] == 10 else x[2:])

In [93]:
reviews.head()

,text,stars,embs,embs_best,embs_worst
user_id,,,,,
-1BSu2dt_rOAqllw9ZDXtA,[Hank and I love Brocatos!The freshest ingredi...,"([0, 1, 2, 3, 4], [6, 9, 11, 0, 1])","[[0.4294569, -0.74091864, 2.5334587, -1.151932...","[[0.4294569, -0.74091864, 2.5334587, -1.151932...","[[0.7082855, -0.7646998, 2.8138785, -1.3410811..."
-6DoXmdXEy_P5N-QZzntgA,"[With a home airport like ORD, SBA is an incre...","([5, 8, 12, 0, 1], [7, 10, 11, 0, 1])","[[0.78651774, -1.0710886, 2.6585214, -1.109733...","[[0.78651774, -1.0710886, 2.6585214, -1.109733...","[[0.4615741, -0.6224606, 2.4306178, -1.099641,..."
-8NOuak4Sipn7-zy7Nk5hg,[One of Philadelphia's best restaurants in my ...,"([0, 1, 3, 4, 5], [11, 8, 6, 2, 12])","[[1.1762301, -1.0045295, 2.9417152, -1.2765405...","[[1.1762301, -1.0045295, 2.9417152, -1.2765405...","[[1.0583673, -0.7620707, 2.6820798, -1.0745525..."
-8rSnT5ztVk6vmTDkxTqsQ,[Great space! Loved the accessibility to outdo...,"([10, 0, 1, 2, 4], [3, 9, 11, 14, 5])","[[1.127403, -0.9143411, 2.9474373, -1.2328093,...","[[1.127403, -0.9143411, 2.9474373, -1.2328093,...","[[0.57977533, -0.7533637, 2.3341944, -1.041487..."
-C7xxeVQI5qEZGAzFdx-cg,[This place is the best! Their food isn't spic...,"([0, 2, 3, 5, 7], [4, 9, 1, 6, 0])","[[0.9144162, -0.76526994, 2.5454423, -1.262568...","[[0.9144162, -0.76526994, 2.5454423, -1.262568...","[[1.2083615, -0.67561406, 2.804841, -1.2537695..."


In [94]:
# gerando media dos embeddings
reviews['embs_best'] = reviews.embs_best.apply(lambda x: np.mean(x, axis=0))
reviews['embs_worst'] = reviews.embs_worst.apply(lambda x: np.mean(x, axis=0))

In [97]:
reviews.embs_best.apply(len).describe()

count    1000.0
mean      768.0
std         0.0
min       768.0
25%       768.0
50%       768.0
75%       768.0
max       768.0
Name: embs_best, dtype: float64

In [98]:
# salvando em parquet os resultados
reviews[['embs_best', 'embs_worst']].to_parquet('../data/embeddingsUsers.parquet')